In [80]:
# Bayesian Network for the Monty Hall problem.
# The DAG is composed by 3 nodes: Guestdoor, Pricedoor, Montydoor. Once the guest chose a door, 
# Monty opens a door (Montydoor) which does not contain the prize (Prizedoor) nor is the door chosen by the guest (Guestdoor).
# We have then Guestdoor and Pricedoor as random, independent variables and Montydoor on the bottom 
# depending on both above nodes.

In [81]:
import numpy as np
from pomegranate import *

In [82]:
# Both Guestdoor and Prizedoor are random:
guest =DiscreteDistribution({ 'A': 1./3, 'B': 1./3, 'C': 1./3 })
prize =DiscreteDistribution({ 'A': 1./3, 'B': 1./3, 'C': 1./3 })

In [83]:
# We can manually build the simple CPT for all the possible combination:
# Let us consider just the first 9 brackets in which the guest chose door A:
# In the first 3 brackets, the prize also is behind door A (second element of each bracket), therefore Monty has 50%
# probability to reveal B or C (and 0% to reveal A because it's forbidden).
# If the prize is behind door B (brackets from 4 to 6), Monty can oly reveal door C (100%, sixth bracket) because
# he cannot reval the door chosen by the guest nor the door with the prize, etc...
monty = ConditionalProbabilityTable(
        [['A', 'A', 'A', 0.0],
         ['A', 'A', 'B', 0.5],
         ['A', 'A', 'C', 0.5],
         ['A', 'B', 'A', 0.0],
         ['A', 'B', 'B', 0.0],
         ['A', 'B', 'C', 1.0],
         ['A', 'C', 'A', 0.0],
         ['A', 'C', 'B', 1.0],
         ['A', 'C', 'C', 0.0],
         ['B', 'A', 'A', 0.0],
         ['B', 'A', 'B', 0.0],
         ['B', 'A', 'C', 1.0],
         ['B', 'B', 'A', 0.5],
         ['B', 'B', 'B', 0.0],
         ['B', 'B', 'C', 0.5],
         ['B', 'C', 'A', 1.0],
         ['B', 'C', 'B', 0.0],
         ['B', 'C', 'C', 0.0],
         ['C', 'A', 'A', 0.0],
         ['C', 'A', 'B', 1.0],
         ['C', 'A', 'C', 0.0],
         ['C', 'B', 'A', 1.0],
         ['C', 'B', 'B', 0.0],
         ['C', 'B', 'C', 0.0],
         ['C', 'C', 'A', 0.5],
         ['C', 'C', 'B', 0.5],
         ['C', 'C', 'C', 0.0]], [guest, prize])

In [84]:
# Instanciating the nodes (state and nodes are the same object. Hidden Markov modelists prefer states)
s1 = Node(guest, name="guest")
s2 = Node(prize, name="prize")
s3 = Node(monty, name="monty")

In [85]:
# Building the model
model = BayesianNetwork("Monty Hall")
model.add_states(s1, s2, s3)
model.add_edge(s1, s3)
model.add_edge(s2, s3)
model.bake()

In [86]:
# Let's say the guest choose door A. We can predict all the probabilites given this evidence:
beliefs = model.predict_proba({ 'guest' : 'A' })
for i,b in enumerate(beliefs):
    print(model.states[i].name)
    print(b.parameters) if hasattr(b, 'parameters') else print(b)
    print("")
# The prize has 33% chance to be behind any of the 3 doors and Monty has to choose between door B and C because
# he (and us) doesn't know where the prize is

guest
A

prize
[{'A': 0.3333333333333333, 'B': 0.3333333333333333, 'C': 0.3333333333333333}]

monty
[{'C': 0.49999999999999994, 'B': 0.49999999999999994, 'A': 0.0}]



In [87]:
# But in the quiz, Monty always know where the prize is and he always reveal a door without the prize
# asking to the guest if he wants to change his initial choice.
# Let's simulate this scenario:
beliefs = model.predict_proba({ 'guest' : 'A', 'monty' : 'B' })
for i,b in enumerate(beliefs):
    print(model.states[i].name)
    print(b.parameters) if hasattr(b, 'parameters') else print(b)
    print("")

guest
A

prize
[{'A': 0.3333333333333334, 'B': 0.0, 'C': 0.6666666666666664}]

monty
B



In [88]:
# We get the well known solution to the game: There's 2/3 chance to get the car by switching door
# while there's 1/3 chance if we keep the initial choice